# Paleobathymetry through time

Create paleobathymetry using combination of [pybacktrack](https://github.com/EarthByte/pyBacktrack) for preserved regions and 'traditionally' made grids for synthetically reconstructed oceanic crust.

For convenience, using traditionally reconstructed grids from Wright et al. 2020: i.e., paleoage grids --> depth to basement using 'GDH1' age-depth relationship (Stein and Stein 1992), [sediment thickness](https://github.com/EarthByte/predicting-sediment-thickness) relationship from Dutkiewicz et al. 2017; and model of large igneous provinces (LIPs) [though won't see these since they only reconstruct on preserved oceanic crust anyway].

This is all done with the Müller et al. (2019) plate reconstruction model.

In [34]:
import gplately
from gplately import pygplates
import matplotlib.pyplot as plt
import numpy as np
import pybacktrack
import pygmt
from plate_model_manager import PlateModelManager
import os
import requests
import multiprocessing
import math
from functools import partial

## Download plate model files

In [3]:
pm_manager = PlateModelManager()
model_name = "Muller2019"
data_dir = "plate-model-repo"

plate_model = pm_manager.get_model(model_name, data_dir=data_dir)

rotation_model = plate_model.get_rotation_model()
topology_filename = plate_model.get_topologies()
static_polygons = plate_model.get_static_polygons()

topology_features_m19 = pygplates.FeatureCollection()
for file in topology_features_m19:
    topology_features_m19 = pygplates.FeatureCollection(file)
    topology_features_m19.add(topology_features_m19)
    
coastlines = plate_model.get_layer('Coastlines')
continents = plate_model.get_layer('ContinentalPolygons')
COBs =  plate_model.get_layer('COBs')

model = gplately.PlateReconstruction(rotation_model, topology_features=topology_features_m19, static_polygons=static_polygons)

downloading https://repo.gplates.org/webdav/pmm/muller2019/Rotations.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/Topologies.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/StaticPolygons.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/Coastlines.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/ContinentalPolygons.zip
The local file(s) is/are still good. Will not download again at this moment.
downloading https://repo.gplates.org/webdav/pmm/muller2019/COBs.zip
The local file(s) is/are still good. Will not download again at this moment.


## Get age grids

In [4]:
%%capture
plate_model.get_rasters("AgeGrids", list(np.arange(0,251,1)))

downloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.0_AgeGrid-0.ncdownloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.0_AgeGrid-1.nc

downloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.0_AgeGrid-2.nc
downloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.0_AgeGrid-3.nc
downloading https://www.earthbyte.org/webdav/ftp/Data_Collections/Muller_etal_2019_Tectonics/Muller_etal_2019_Agegrids/Muller_etal_2019_Tectonics_v2.0_netCDF/Muller_etal_2019_Tectonics_v2.

In [5]:
agegrid_filename_prefix = '%s/%s/Rasters/AgeGrids/Muller_etal_2019_Tectonics_v2.0_AgeGrid-' % (data_dir, model_name)
agegrid_filename_ext = '.nc'

### Download the Wright et al. 2020 paleobathymetry grids

These use Müller et al. 2019 as their plate reconstruction

In [6]:
# create output directory for paleobathymetry
pbathy_dir = '%s/%s/Rasters/paleobathymetry/Grids-Muller_etal_2019_v2.0/' % (data_dir, model_name)
os.makedirs(pbathy_dir, exist_ok=True)

In [7]:
# folder where my paleobathymetry in the Muller et al. 2019 plate model is
# this is as part of the supplementary data to Wright et al. 2020

pbathy_folder = 'https://www.earthbyte.org/webdav/ftp/Data_Collections/Wright_etal_2020_ESR/Grids-Muller_etal_2019_v2.0/Paleobathymetry_GDH1-Muller++_2019_v2.0/'

In [9]:
# set up the times we want to download the paleobathymetry netcdfs for - maybe you don't want all of them!
times_pbathy = np.arange(0, 201, 1)

for time in times_pbathy:
    with open('%s/paleobathymetry_%sMa.nc' % (pbathy_dir, time), 'wb') as out_file:
        content = requests.get('%s/paleobathymetry_%sMa.nc' % (pbathy_folder, time), allow_redirects=True,).content
        out_file.write(content)

## Run pybacktrack

In [10]:
# set all the pybacktrack parameters
pybacktrack_output_dir = '%s/%s/Rasters/paleobathymetry-pybacktrack/' % (data_dir, model_name)
os.makedirs(pybacktrack_output_dir, exist_ok=True)

In [19]:
grid_spacing = 0.1
py_age_depth_model = pybacktrack.AGE_TO_DEPTH_MODEL_GDH1
max_time = 200
rotation_timestep = 1
anchor_plate_id = 0
dynamic_topography_model_name = None

num_cpus = 6
max_num_cpus = multiprocessing.cpu_count()

# this is so I don't accidentally try and use too many CPUs!
if num_cpus > max_num_cpus:
    print('%s CPUs requested, while only %s are available' % (num_cpus, max_num_cpus))
    print('Using %s CPUs instead' % (max_num_cpus - 1))
    num_cpus = max_num_cpus - 1

In [20]:
pybacktrack.reconstruct_paleo_bathymetry_grids(
    output_file_prefix='%s/paleobathymetry' % pybacktrack_output_dir,
    grid_spacing_degrees=grid_spacing,
    ocean_age_to_depth_model=py_age_depth_model,
    age_grid_filename = '%s0%s' % (agegrid_filename_prefix, agegrid_filename_ext), # just want present day agegrid
    static_polygon_filename=static_polygons,
    rotation_filenames=rotation_model,
    oldest_time=max_time,
    time_increment=rotation_timestep,
    use_all_cpus=num_cpus, anchor_plate_id=anchor_plate_id)

grdtrack [WARNING]: Some input points were outside the grid domain(s).


### Merge paleobathymetry

Folloing code is essentially copy/pasted from a pybacktrack merge utils code

In [42]:
# Merged grid name
merged_grid_directory = '%s/merged/' % pybacktrack_output_dir
merged_grid_basename = 'paleobathymetry_'
os.makedirs(merged_grid_directory, exist_ok=True)
lon_min = -180
lon_max = 180
lat_min = -90
lat_max = 90


# PyBacktrack paleobathymetry grids.
paleo_bathymetry_pybacktrack_prefix = pybacktrack_output_dir
paleo_bathymetry_pybacktrack_basename = 'paleobathymetry'
paleo_bathymetry_pybacktrack_extension = 'nc'
# Typically either 0 or 1.
paleo_bathymetry_pybacktrack_decimal_places_in_time = 1

# Traditional paleobathymetry grids.
paleo_bathymetry_wright_prefix = pbathy_dir
paleo_bathymetry_wright_basename = 'paleobathymetry'
paleo_bathymetry_wright_extension = 'Ma.nc'
# Typically either 0 or 1.
paleo_bathymetry_wright_decimal_places_in_time = 0

merged_grid_spacing_degrees = 0.1

In [43]:
# Create the dynamic topography model (from model name) if requested.
if dynamic_topography_model_name:
    dynamic_topography_model = pybacktrack.InterpolateDynamicTopography.create_from_bundled_model(dynamic_topography_model_name)
else:
    dynamic_topography_model = None

def merge_paleo_bathymetry_grids(
        time,
        input_points,
        dynamic_topography_at_present_day):
    print('... Merging grids at %s Ma' % time)
    # Paleo bathymetry grids to merge (pybacktrack and Wright).
    # Note that pybacktrack uses 1 decimal place for the time in the filename, whereas Wright does not.
    paleo_bathymetry_pybacktrack_filename = os.path.join(
            paleo_bathymetry_pybacktrack_prefix,
            '{0}_{1:.{2}f}.{3}'.format(paleo_bathymetry_pybacktrack_basename, time, paleo_bathymetry_pybacktrack_decimal_places_in_time, paleo_bathymetry_pybacktrack_extension))

    paleo_bathymetry_wright_filename = os.path.join(paleo_bathymetry_wright_prefix,
        '{0}_{1:.{2}f}{3}'.format(paleo_bathymetry_wright_basename, time, paleo_bathymetry_wright_decimal_places_in_time, paleo_bathymetry_wright_extension))

    # Sample the paleo bathymetry grids that we're going to merge.
    # print('Reading input bathymetry grids...'); sys.stdout.flush()
    paleo_bathymetry_points = _load_bathymetry(
            input_points,
            paleo_bathymetry_pybacktrack_filename,
            paleo_bathymetry_wright_filename)
    
    if dynamic_topography_model:
        # Sample the dynamic topography at 'time'.
        # print('Sample dynamic topography at {}...'.format(time)); sys.stdout.flush()
        dynamic_topography = dynamic_topography_model.sample(time, input_points)
    
    # print('Merging...'); sys.stdout.flush()
    merged_points = []
    for point_index, paleo_bathymetry_point in enumerate(paleo_bathymetry_points):
        lon, lat, paleo_bathymetry_pybacktrack, paleo_bathymetry_wright = paleo_bathymetry_point
        if math.isnan(paleo_bathymetry_pybacktrack) and math.isnan(paleo_bathymetry_wright):
            # Skip point if no paleo bathymetry from pybacktrack or Wright.
            continue
        
        # Prefer pybacktrack paleobathymetry.
        if not math.isnan(paleo_bathymetry_pybacktrack):
            paleo_bathymetry = paleo_bathymetry_pybacktrack
        else:
            # Note that pybacktrack generates paleobathymetry grids with negative values below sea level by default
            # (the opposite of backtracking which outputs positive depths below sea level).
            # And this matches the Wright paleobathymetry grids (which also have negative values below sea level),
            # so we don't need to negate them to match pybacktrack-generated paleobathymetry.
            paleo_bathymetry = paleo_bathymetry_wright
            # Also apply dynamic topography to Wright grids if requested (pybacktrack already has it applied).
            if dynamic_topography_model:
                # Dynamic topography, like bathymetry, is positive going up and negative going down so we can just add it to bathymetry.
                paleo_bathymetry += dynamic_topography[point_index] - dynamic_topography_at_present_day[point_index]

        merged_points.append((lon, lat, paleo_bathymetry))
    

    merged_grid_filename = os.path.join(merged_grid_directory, '{0}{1}Ma.nc'.format(merged_grid_basename, time))
    _gmt_nearneighbor(merged_points, merged_grid_spacing_degrees, merged_grid_filename, lon_min, lon_max, lat_min, lat_max)



def _load_bathymetry(input_points, paleo_bathymetry_pybacktrack_filename, paleo_bathymetry_wright_filename):

    # Use the Python xarray module (if installed) to load the bathymetry grids.
    #
    # This enables faster loading of the grids (compared to GMT grdtrack) but any
    # interpolated location that has any NaN neighbours (in 4x4 linear region) will itself be NaN
    # (whereas GMT grdtrack will interpolate halfway from a non-NaN value, using option "-n+t0.5").
    #
    # UPDATE: No longer using xarray since:
    #         - xarray seems to require the longitude range of the sampling points to match the input grid (whereas GMT grdtrack does not).
    #           For example, if sampling points are [-180, 180] and grid is [0, 360] then an entire hemisphere goes missing.
    #         - xarray appears to shift away from NaN regions slightly (likely due to the NaN interpolation method covered above).
    #
    use_xarray_if_available = False
    
    if use_xarray_if_available and have_xarray:

        # The input point longitudes and latitudes.
        lons = xarray.DataArray([lon for lon, lat in input_points])
        lats = xarray.DataArray([lat for lon, lat in input_points])

        with xarray.open_dataset(paleo_bathymetry_pybacktrack_filename) as bathymetry_pybacktrack_grid:
            # Get the lon, lat coordinates names (typically 'lon' and 'lat', and in that order).
            coord_names = list(bathymetry_pybacktrack_grid.coords.keys())
            lon_name = coord_names[0]
            lat_name = coord_names[1]
            # Get the z value name (typically 'z').
            z_name = list(bathymetry_pybacktrack_grid.data_vars.keys())[0]
            # Interpolate the bathymetry grid at the input point locations.
            bathymetry_pybacktrack_values = bathymetry_pybacktrack_grid[z_name].interp(dict({lon_name : lons, lat_name : lats}))
            # print(bathymetry_pybacktrack_grid.keys())
            # print(bathymetry_pybacktrack_values.shape)

        with xarray.open_dataset(paleo_bathymetry_wright_filename) as bathymetry_wright_grid:
            # Get the lon, lat coordinates names (typically 'lon' and 'lat', and in that order).
            coord_names = list(bathymetry_wright_grid.coords.keys())
            lon_name = coord_names[0]
            lat_name = coord_names[1]
            # Get the z value name (typically 'z').
            z_name = list(bathymetry_wright_grid.data_vars.keys())[0]
            # Interpolate the bathymetry grid at the input point locations.
            bathymetry_wright_values = bathymetry_wright_grid[z_name].interp(dict({lon_name : lons, lat_name : lats}))
            # print(bathymetry_wright_grid.keys())
            # print(bathymetry_wright_values.shape)

        output_values = []

        for point_index in range(len(lons)):
            lon, lat = input_points[point_index]
            output_value = (lon, lat, float(bathymetry_pybacktrack_values.item(point_index)), float(bathymetry_wright_values.item(point_index)))
            output_values.append(output_value)
    
        return output_values

    else:  # Use GMT grdtrack ...

        # Create a multiline string (one line per lon/lat/value1/etc row).
        location_data = ''.join(
                ' '.join(str(item) for item in row) + '\n' for row in input_points)

        # The command-line strings to execute GMT 'grdtrack'.
        grdtrack_command_line = ["gmt", "grdtrack",
            # Geographic input/output coordinates...
            "-fg",
            # Avoid anti-aliasing...
            "-n+a+bg+t0.5"]
        # One or more grid filenames to sample.
        for grid_filename in (paleo_bathymetry_pybacktrack_filename, paleo_bathymetry_wright_filename):
            grdtrack_command_line.append("-G{0}".format(grid_filename))
        
        # Call the system command.
        stdout_data = pybacktrack.util.call_system_command.call_system_command(grdtrack_command_line, stdin=location_data, return_stdout=True)

        output_values = []

        # Extract the sampled values.
        for line in stdout_data.splitlines():
            # Each line returned by GMT grdtrack contains "longitude latitude grid1_value [grid2_value ...]".
            # Note that if GMT returns "NaN" then we'll return float('nan').
            output_value = tuple(float(value) for value in line.split())
            output_values.append(output_value)
    
        return output_values


def _gmt_nearneighbor(input, grid_spacing_degrees, grid_filename, lon_min, lon_max, lat_min, lat_max):
    
    # Create a multiline string (one line per lon/lat/value row).
    input_data = ''.join(
            ' '.join(str(item) for item in row) + '\n' for row in input)

    # The command-line strings to execute GMT 'nearneighbor'.
    nearneighbor_command_line = [
        "gmt",
        "nearneighbor",
        "-N1/1", # Divide search radius into 1 sector and only require a value in 1 sector.
        "-S{0}d".format(0.1 * grid_spacing_degrees),
        "-I{0}".format(grid_spacing_degrees),
        # Use GMT gridline registration since our input point grid has data points on the grid lines.
        # Gridline registration is the default so we don't need to force pixel registration...
        # "-r", # Force pixel registration since data points are at centre of cells.
        "-R%s/%s/%s/%s" % (lon_min, lon_max, lat_min, lat_max),
        "-fg",
        "-G{0}".format(grid_filename), '--IO_NC4_DEFLATION_LEVEL=9']
    
    # Call the system command.
    pybacktrack.util.call_system_command.call_system_command(nearneighbor_command_line, stdin=input_data)


def _generate_input_points_grid(grid_spacing_degrees):
    """
    Generate a global grid of points uniformly spaced in latitude and longitude.

    Returns a list of (longitude, latitude) tuples.
    """
    
    if grid_spacing_degrees == 0:
        raise ValueError('Grid spacing cannot be zero.')
    
    input_points = []
    
    # Data points start *on* dateline (-180).
    # If 180 is an integer multiple of grid spacing then final longitude also lands on dateline (+180).
    num_latitudes = int(math.floor(180.0 / grid_spacing_degrees)) + 1
    num_longitudes = int(math.floor(360.0 / grid_spacing_degrees)) + 1
    for lat_index in range(num_latitudes):
        lat = -90 + lat_index * grid_spacing_degrees
        
        for lon_index in range(num_longitudes):
            # NOTE: For some reason xarray does not always like the range [-180, 180] (excludes half the globe).
            #       So use the range [0, 360] instead.
            lon = 0 + lon_index * grid_spacing_degrees
            
            input_points.append((lon, lat))
    
    return input_points

In [45]:
%%capture
# Generate a global latitude/longitude grid of points (with the requested grid spacing).
input_points = _generate_input_points_grid(merged_grid_spacing_degrees)

if dynamic_topography_model:
    # Sample the dynamic topography at present day.
    # print('Sample dynamic topography at {}...'.format(0)); sys.stdout.flush()
    dynamic_topography_at_present_day = dynamic_topography_model.sample(0, input_points)
else:
    dynamic_topography_at_present_day = None

# Create times from present day to 'max_time'.
time_range = np.arange(0, max_time + rotation_timestep, rotation_timestep)

for time in time_range:
    merge_paleo_bathymetry_grids(
            time,
            input_points,
            dynamic_topography_at_present_day)

### Plot paleobathymetry

In [46]:
paleobathymetry_filename_prefix = '%s/%s' % (merged_grid_directory, merged_grid_basename)
paleobathymetry_filename_ext = 'Ma.nc'

In [47]:
times = np.arange(0, max_time + rotation_timestep, rotation_timestep)

In [48]:
def plot_animation_through_time(time, plate_model, outdir, grid_filename_prefix, grid_filename_ext, central_meridian):

    age = time

    gplot = gplately.PlotTopologies(plate_model,
                                    coastlines=coastlines,
                                    continents=continents,
                                    COBs=COBs,
                                    time=age)


    grid = '%s%s%s' % (grid_filename_prefix, int(time), grid_filename_ext)

    # ----- parameters for plot
    region = 'd'
   
    width = 12
    projection = 'N%s/' % central_meridian
    
    # plate boundary stuff
    plateboundary_width = '0.5p'
    plate_colour = 'black'
    subduction_zone_colour = 'dodgerblue3'
    ridge_colour = 'red'
    
    age_font = '14p,Helvetica,black'
    coastline_col = 'grey80'
    continent_col='grey70'
    
    label_font = '14p,Helvetica,black'
    label_offset = 'J0.1/0.0'
    label_position = 'TL'
    
    # ----- get plot things for gplately
    gdf_subduction_left, gdf_subduction_right = gplot.get_subduction_direction()
    
    # resolve plates - get topologies and subset to only closed plate boundaries
    gdf_topo = gplot.get_all_topologies(central_meridian=central_meridian)
    gdf_topo_plates = gdf_topo[(gdf_topo['feature_name'] == 'TopologicalClosedPlateBoundary')]
    gdf_topo_plates = gdf_topo_plates[~gdf_topo_plates.is_empty]

    # get ACTUAL transforms
    gdf_topo_sections = gplot.get_all_topological_sections()
    gdf_topo_transforms = gdf_topo_sections[(gdf_topo_sections['feature_type'] == pygplates.FeatureType.gpml_transform)]
    gdf_topo_transforms = gdf_topo_transforms.drop(columns='feature_type')  # get rid of the feature type column otherwise it will complain when plotting

    # -------------------------------------------------------------------------------
    # ------ plot
    fig = pygmt.Figure()
    pygmt.config(FONT_ANNOT=10, FONT_LABEL=8, FONT=10, MAP_TICK_PEN="0.75p", MAP_FRAME_PEN="0.75p", MAP_TICK_LENGTH_PRIMARY="4p")
        
    pygmt.makecpt(cmap="deep", series=[-6500, 0], reverse=True, background=True)
    fig.grdimage(grid=grid, region=region, projection="%s%sc" % (projection, width))
    fig.colorbar(frame=["xa1000f200+lElevation (m)"], position="JMR+o0.5c/0.2c+w5.25c/0.3c+eb+n")
     
    fig.plot(data=gplot.get_coastlines(), fill=coastline_col, transparency=0) # COBs

    fig.basemap(region=region, projection="%s%sc" % (projection, width), frame="lrtb")
    fig.text(text='%s Ma' % age, position=label_position, no_clip=True, font=label_font, offset=label_offset)
    
    fig.savefig('%s/pbathy-%sMa.png' % (outdir, age), dpi=300)

In [49]:
plate_model = model
central_meridian = 180

In [50]:
plot_type = 'pbathy_robinson'
outdir = 'figures_%s_%s' % (model_name, plot_type)
os.makedirs(outdir, exist_ok=True)

# plot files
for time in times:
    plot_animation_through_time(time, plate_model, outdir, paleobathymetry_filename_prefix, paleobathymetry_filename_ext, central_meridian)

In [51]:
# copy 0 Ma
os.system('cp %s/pbathy-0Ma.png %s/pbathy0Ma.png' % (outdir, outdir))
os.system('ffmpeg -r 12 -start_number -%s -i %s/pbathy%sMa.png  -pix_fmt yuv420p -vf "split=2[clr][bg];[bg]drawbox=c=white:t=fill[bg];[bg][clr]overlay,scale=trunc(iw/3.5)*2:trunc(ih/3.5)*2,pad=width=iw+50:height=ih+50:x=(ow-iw)/2:y=(oh-ih)/2:color=white" -y %s_pbathy.mp4 < /dev/null'
 % (max_time, outdir, '%d', model_name))

ffmpeg version 6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with clang version 15.0.7
  configuration: --prefix=/Users/runner/miniforge3/conda-bld/ffmpeg_1692995812302/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl --cc=arm64-apple-darwin20.0.0-clang --cxx=arm64-apple-darwin20.0.0-clang++ --nm=arm64-apple-darwin20.0.0-nm --ar=arm64-apple-darwin20.0.0-ar --disable-doc --disable-openssl --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libopenh264 --enable-libdav1d --enable-cross-compile --arch=arm64 --target-os=darwin --cross-prefix=arm64-apple-darwin20.0.0- --host-cc=/Users/runner/miniforge3/conda-bld/ffmpeg_1692995812302/_build_env/bin/x86_64-apple-darwin13.4.0-clang --enable-neon --enable-gnutls --enable-libmp3lame --enable-libvpx --enable-libass --enable-pthreads --enab

0